In [15]:
import warnings
warnings.filterwarnings('ignore')
from math import floor
import requests
import time
import base64
import hashlib
import hmac
import json
import urllib.request as urllib2
import ssl
import pandas as pd
import ast
import datetime
import urllib.parse
import hashlib
import hmac
import base64
import time

class API(object):
    
    def __init__(self, apiPath, apiPublicKey="", apiPrivateKey="", timeout=10, checkCertificate=True, useNonce=False, apiversion='0'):
        self.apiPath = apiPath
        self.apiPublicKey = apiPublicKey
        self.apiPrivateKey = apiPrivateKey
        self.timeout = timeout
        self.nonce = 0
        self.apiversion = apiversion
        self.session = requests.Session()
        self.checkCertificate = checkCertificate
        self.useNonce = useNonce
        self.response = None
        self._json_options = {}
        
    def get_apiversion(self):
        return self.apiversion
    
    def json_options(self, **kwargs):
        """ Set keyword arguments to be passed to JSON deserialization.
        :param kwargs: passed to :py:meth:`requests.Response.json`
        :returns: this instance for chaining
        """
        self._json_options = kwargs
        return self

    def close(self):
        """ Close this session.
        :returns: None
        """
        self.session.close()
        return

    def load_key(self, path):
        """ Load key and secret from file.
        Expected file format is key and secret on separate lines.
        :param path: path to keyfile
        :type path: str
        :returns: None
        """
        with open(path, 'r') as f:
            self.apiPublicKey = f.readline().strip()
            self.apiPrivateKey = f.readline().strip()
        return

    def _query(self, urlpath, data, headers=None, timeout=None):
        """ Low-level query handling.
        .. note::
           Use :py:meth:`query_private` or :py:meth:`query_public`
           unless you have a good reason not to.
        :param urlpath: API URL path sans host
        :type urlpath: str
        :param data: API request parameters
        :type data: dict
        :param headers: (optional) HTTPS headers
        :type headers: dict
        :param timeout: (optional) if not ``None``, a :py:exc:`requests.HTTPError`
                        will be thrown after ``timeout`` seconds if a response
                        has not been received
        :type timeout: int or float
        :returns: :py:meth:`requests.Response.json`-deserialised Python object
        :raises: :py:exc:`requests.HTTPError`: if response status not successful
        """
        if data is None:
            data = {}
        if headers is None:
            headers = {}

        url = self.apiPath + urlpath

        self.response = self.session.post(url, data = data, headers = headers,
                                          timeout = timeout)

        return self.response.json(**self._json_options)
    
#%%
class API_spot(API):   
    def __init__(self, apiPublicKey="", apiPrivateKey="", timeout=10, checkCertificate=True, useNonce=False):
        super().__init__('https://api.kraken.com', apiPublicKey, apiPrivateKey, timeout, checkCertificate, useNonce,'0')
    
    def query_public(self, method, data=None, timeout=None):
        """ Performs an API query that does not require a valid key/secret pair.
        :param method: API method name
        :type method: str
        :param data: (optional) API request parameters
        :type data: dict
        :param timeout: (optional) if not ``None``, a :py:exc:`requests.HTTPError`
                        will be thrown after ``timeout`` seconds if a response
                        has not been received
        :type timeout: int or float
        :returns: :py:meth:`requests.Response.json`-deserialised Python object
        """
        if data is None:
            data = {}

        urlpath = '/' + self.apiversion + '/public/' + method
        
        return self._query(urlpath, data, timeout = timeout)
    
    def query_private(self, method, data=None, timeout=None):
        """ Performs an API query that requires a valid key/secret pair.
        :param method: API method name
        :type method: str
        :param data: (optional) API request parameters
        :type data: dict
        :param timeout: (optional) if not ``None``, a :py:exc:`requests.HTTPError`
                        will be thrown after ``timeout`` seconds if a response
                        has not been received
        :type timeout: int or float
        :returns: :py:meth:`requests.Response.json`-deserialised Python object
        """
        if data is None:
            data = {}

        if not self.apiPublicKey or not self.apiPrivateKey:
            raise Exception('Either key or secret is not set! (Use `load_key()`.')

        data['nonce'] = self._nonce()

        urlpath = '/' + self.apiversion + '/private/' + method

        headers = {
            'API-Key': self.apiPublicKey,
            'API-Sign': self._sign(data, urlpath)
        }

        return self._query(urlpath, data, headers, timeout = timeout)
    
    def _nonce(self):
        """ Nonce counter.
        :returns: an always-increasing unsigned integer (up to 64 bits wide)
        """
        return int(1000*time.time())

    def _sign(self, data, urlpath):
        """ Sign request data according to Kraken's scheme.
        :param data: API request parameters
        :type data: dict
        :param urlpath: API URL path sans host
        :type urlpath: str
        :returns: signature digest
        """
        postdata = urllib.parse.urlencode(data)

        # Unicode-objects must be encoded before hashing
        encoded = (str(data['nonce']) + postdata).encode()
        message = urlpath.encode() + hashlib.sha256(encoded).digest()

        signature = hmac.new(base64.b64decode(self.apiPrivateKey),
                             message, hashlib.sha512)
        sigdigest = base64.b64encode(signature.digest())

        return sigdigest.decode()
    
    def get_bid(self,ticker):
        tickerinfo = self.query_public("Ticker", {"pair":ticker})
        bid = tickerinfo['result'][ticker]['b'][0]
        return float(bid)
    
    def get_ask(self,ticker):
        tickerinfo = self.query_public("Ticker", {"pair":ticker})
        ask = tickerinfo['result'][ticker]['a'][0]
        return float(ask)
    
    def get_mid(self,ticker):
        bid = self.get_bid(ticker)
        ask = self.get_ask(ticker)
        mid=(bid+ask)/2
        return mid
    
    def get_balance(self,ccy):
        return self.query_private('Balance')['result'][ccy]
    
    def create_order(self,ticker,side,orderType,price,volume):
        response = self.query_private('AddOrder',
                       {'pair':ticker,
                        'type':side,
                        'ordertype':orderType,
                        'price':price,
                        'volume':volume,
                        })
        return response
    
    def transfer_from_spot_to_future(self, ccy, amount):
        response = self.query_private('WalletTransfer', {'asset':ccy,
                                          'from':'Spot Wallet',
                                          'to':'Futures Wallet',
                                          'amount':amount})
        return response

class API_future(API):
    def __init__(self, apiPublicKey="", apiPrivateKey="", timeout=10, checkCertificate=True, useNonce=False):
        super().__init__('https://futures.kraken.com/derivatives', apiPublicKey, apiPrivateKey, timeout, checkCertificate, useNonce,'api/v3')
        self.apiPath ="https://www.cryptofacilities.com/derivatives"
    
    def _nonce(self):
        """ Nonce counter.
        :returns: an always-increasing unsigned integer (up to 64 bits wide)
        """
        return int(1000*time.time())
    
    def query_public(self, method, data=None, timeout=None):
        """ Performs an API query that does not require a valid key/secret pair.
        :param method: API method name
        :type method: str
        :param data: (optional) API request parameters
        :type data: dict
        :param timeout: (optional) if not ``None``, a :py:exc:`requests.HTTPError`
                        will be thrown after ``timeout`` seconds if a response
                        has not been received
        :type timeout: int or float
        :returns: :py:meth:`requests.Response.json`-deserialised Python object
        """
        if data is None:
            data = {}

        urlpath = '/' + self.apiversion + "/"+ method + "?" + "symbol=%s" % data["pair"]
        #return urlpath
        return self._query(urlpath, data, timeout = timeout)
    
    def query_private(self, method, data=None, timeout=None):
        """ Performs an API query that requires a valid key/secret pair.
        :param method: API method name
        :type method: str
        :param data: (optional) API request parameters
        :type data: dict
        :param timeout: (optional) if not ``None``, a :py:exc:`requests.HTTPError`
                        will be thrown after ``timeout`` seconds if a response
                        has not been received
        :type timeout: int or float
        :returns: :py:meth:`requests.Response.json`-deserialised Python object
        """
        if data is None:
            data = {}

        if not self.apiPublicKey or not self.apiPrivateKey:
            raise Exception('Either key or secret is not set! (Use `load_key()`.')

        data['nonce'] = self._nonce()

        urlpath = '/' + self.apiversion + '/' + method

        headers = {
            'API-Key': self.apiPublicKey,
            'API-Sign': self.sign_message(urlpath)
        }

        return self._query(urlpath, data, headers, timeout = timeout)
    
    def get_instruments(self):
        endpoint = "/api/v3/instruments"
        return self.make_request("GET", endpoint)

    # returns market data for all instruments
    def get_tickers(self):
        endpoint = "/api/v3/tickers"
        return self.make_request("GET", endpoint)

    # returns the entire order book of a futures
    def get_orderbook(self, symbol):
        endpoint = "/api/v3/orderbook"
        postUrl = "symbol=%s" % symbol
        return self.make_request("GET", endpoint, postUrl=postUrl)

    # returns historical data for futures and indices
    def get_history(self, symbol, lastTime=""):
        endpoint = "/api/v3/history"
        if lastTime != "":
            postUrl = "symbol=%s&lastTime=%s" % (symbol, lastTime)
        else:
            postUrl = "symbol=%s" % symbol
        return self.make_request("GET", endpoint, postUrl=postUrl)
    

    ##### private endpoints #####

    # returns key account information
    # Deprecated because it returns info about the Futures margin account
    # Use get_accounts instead
    def get_account(self):
        endpoint = "/api/v3/account"
        return self.make_request("GET", endpoint)

    # returns key account information
    def get_accounts(self):
        endpoint = "/api/v3/accounts"
        return self.make_request("GET", endpoint)

    # places an order
    def send_order(self, orderType, symbol, side, size, limitPrice, stopPrice=None, clientOrderId=None):
        endpoint = "/api/v3/sendorder"
        postBody = "orderType=%s&symbol=%s&side=%s&size=%s&limitPrice=%s" % (orderType, symbol, side, size, limitPrice)

        if orderType == "stp" and stopPrice is not None:
            postBody += "&stopPrice=%s" % stopPrice

        if clientOrderId is not None:
            postBody += "&cliOrdId=%s" % clientOrderId

        return self.make_request("POST", endpoint, postBody=postBody)

    # places an order
    def send_order_1(self, order):
        endpoint = "/api/v3/sendorder"
        postBody = urllib.parse.urlencode(order)
        return self.make_request("POST", endpoint, postBody=postBody)

    # edit an order
    def edit_order(self, edit):
        endpoint = "/api/v3/editorder"
        postBody = urllib.parse.urlencode(edit)
        return self.make_request("POST", endpoint, postBody=postBody)

    # cancels an order
    def cancel_order(self, order_id=None, cli_ord_id=None):
        endpoint = "/api/v3/cancelorder"

        if order_id is None:
            postBody = "cliOrdId=%s" % cli_ord_id
        else:
            postBody = "order_id=%s" % order_id

        return self.make_request("POST", endpoint, postBody=postBody)

    # cancel all orders
    def cancel_all_orders(selfs, symbol=None):
        endpoint = "/api/v3/cancelallorders"
        if symbol is not None:
            postbody = "symbol=%s" % symbol
        else:
            postbody = ""

        return selfs.make_request("POST", endpoint, postBody=postbody)

    # cancel all orders after
    def cancel_all_orders_after(selfs, timeoutInSeconds=60):
        endpoint = "/api/v3/cancelallordersafter"
        postbody = "timeout=%s" % timeoutInSeconds

        return selfs.make_request("POST", endpoint, postBody=postbody)

    # places or cancels orders in batch
    def send_batchorder(self, jsonElement):
        endpoint = "/api/v3/batchorder"
        postBody = "json=%s" % jsonElement
        return self.make_request("POST", endpoint, postBody=postBody)

    # returns all open orders
    def get_openorders(self):
        endpoint = "/api/v3/openorders"
        return self.make_request("GET", endpoint)

    # returns filled orders
    def get_fills(self, lastFillTime=""):
        endpoint = "/api/v3/fills"
        if lastFillTime != "":
            postUrl = "lastFillTime=%s" % lastFillTime
        else:
            postUrl = ""
        return self.make_request("GET", endpoint, postUrl=postUrl)

    # returns all open positions
    def get_openpositions(self):
        endpoint = "/api/v3/openpositions"
        return self.make_request("GET", endpoint)

    # return the user recent orders
    def get_recentorders(self, symbol=""):
        endpoint = "/api/v3/recentorders"
        if symbol != "":
            postUrl = "symbol=%s" % symbol
        else:
            postUrl = ""
        return self.make_request("GET", endpoint, postUrl=postUrl)

    # sends an xbt withdrawal request
    def send_withdrawal(self, targetAddress, currency, amount):
        endpoint = "/api/v3/withdrawal"
        postBody = "targetAddress=%s&currency=%s&amount=%s" % (targetAddress, currency, amount)
        return self.make_request("POST", endpoint, postBody=postBody)
    
    def send_withdrawal_to_spot(self, currency, amount):
        endpoint = "/api/v3/withdrawal"
        postBody = "currency=%s&amount=%s" % (currency, amount)
        return self.make_request("POST", endpoint, postBody=postBody)

    # returns xbt transfers
    def get_transfers(self, lastTransferTime=""):
        endpoint = "/api/v3/transfers"
        if lastTransferTime != "":
            postUrl = "lastTransferTime=%s" % lastTransferTime
        else:
            postUrl = ""
        return self.make_request("GET", endpoint, postUrl=postUrl)

    # returns all notifications
    def get_notifications(self):
        endpoint = "/api/v3/notifications"
        return self.make_request("GET", endpoint)

    # makes an internal transfer
    def transfer(self, fromAccount, toAccount, unit, amount):
        endpoint = "/api/v3/transfer"
        postBody = "fromAccount=%s&toAccount=%s&unit=%s&amount=%s" % (fromAccount, toAccount, unit, amount)
        return self.make_request("POST", endpoint, postBody=postBody)

    # signs a message
    def sign_message(self, endpoint, postData, nonce=""):
        # step 1: concatenate postData, nonce + endpoint                
        message = postData + nonce + endpoint

        # step 2: hash the result of step 1 with SHA256
        sha256_hash = hashlib.sha256()
        sha256_hash.update(message.encode('utf8'))
        hash_digest = sha256_hash.digest()

        # step 3: base64 decode apiPrivateKey
        secretDecoded = base64.b64decode(self.apiPrivateKey)

        # step 4: use result of step 3 to has the result of step 2 with HMAC-SHA512
        hmac_digest = hmac.new(secretDecoded, hash_digest, hashlib.sha512).digest()

        # step 5: base64 encode the result of step 4 and return
        return base64.b64encode(hmac_digest)

    # creates a unique nonce
    def get_nonce(self):
        # https://en.wikipedia.org/wiki/Modulo_operation
        self.nonce = (self.nonce + 1) & 8191
        return str(int(time.time() * 1000)) + str(self.nonce).zfill(4)

    # sends an HTTP request
    def make_request(self, requestType, endpoint, postUrl="", postBody=""):
        # create authentication headers
        postData = postUrl + postBody

        if self.useNonce:
            nonce = self.get_nonce()
            signature = self.sign_message(endpoint, postData, nonce=nonce)
            authentHeaders = {"APIKey": self.apiPublicKey, "Nonce": nonce, "Authent": signature}
        else:
            signature = self.sign_message(endpoint, postData)
            authentHeaders = {"APIKey": self.apiPublicKey, "Authent": signature}

        # create request
        url = self.apiPath + endpoint + "?" + postUrl
        request = urllib2.Request(url, str.encode(postBody), authentHeaders)
        request.get_method = lambda: requestType

        # read response
        if self.checkCertificate:
            response = urllib2.urlopen(request, timeout=self.timeout)
        else:
            ctx = ssl.create_default_context()
            ctx.check_hostname = False
            ctx.verify_mode = ssl.CERT_NONE
            response = urllib2.urlopen(request, context=ctx, timeout=self.timeout)

        response = response.read().decode("utf-8")

        # return
        return response
    
    def get_bid(self,ticker):
        orderBook=ast.literal_eval(self.get_orderbook(ticker))
        bids=orderBook["orderBook"]["bids"]
        bid=bids[0][0]
        return bid
    
    def get_ask(self,ticker):
        orderBook=ast.literal_eval(self.get_orderbook(ticker))
        asks=orderBook["orderBook"]["asks"]
        ask=asks[0][0]
        return ask
    
    def get_mid(self,ticker):
        bid = self.get_bid(ticker)
        ask = self.get_ask(ticker)
        mid=(bid+ask)/2
        return mid
    
    def get_ticker_info(self, ticker, info):
        all_tickers_info = pd.DataFrame(ast.literal_eval(self.get_instruments().replace("true","True").replace("false","False"))['instruments'])
        ticker_info = all_tickers_info[all_tickers_info["symbol"]==ticker]
        info = ticker_info[info].values[0]
        return info
    
    def get_time_to_expiry(self,ticker):
        time_to_expiry = abs(pd.datetime.today().day - pd.datetime.strptime(self.get_ticker_info(ticker,"lastTradingTime"),"%Y-%m-%dT%H:%M:%S.%fZ").day)/365
        return time_to_expiry 
    
    def get_ticker(self,future_type,expiry):
        """
        Only 2 types of future:
            - monthly M
            - quaterly Q
        """
        all_tickers_info = pd.DataFrame(ast.literal_eval(self.get_instruments().replace("true","True").replace("false","False"))['instruments'])
        futures = all_tickers_info[all_tickers_info['symbol'].str.contains(future_type)]
        futures['lastTradingTime'] = futures['lastTradingTime'].apply(lambda x: pd.datetime.strptime(x,"%Y-%m-%dT%H:%M:%S.%fZ"))
        if expiry=="M":
            result = futures[futures['lastTradingTime']==min(futures['lastTradingTime'])]['symbol'].values[0]
        elif expiry=="Q":
            result = futures[futures['lastTradingTime']==max(futures['lastTradingTime'])]['symbol'].values[0]
        else:
            raise ValueError('Bad expiry type: "' +expiry+'". Must be "Q" or "M"')
        return result
    
    def get_cash(self, currency):
        accountInfo = ast.literal_eval(self.get_accounts())['accounts']
        return accountInfo['cash']['balances'][currency]
    
    
######################
# SEPARATE FUNCTIONS #
######################

def get_best_opportunity_ticker_arb_future_spot():
    best_performing_sym=""
    max_perf=-1
    tickers = get_all_future_tickers()
    for ticker in tickers:
        try:
            perf = compute_spot_fut_arb_performance_monthly(ticker, 1e3,debug=True)
            if perf>max_perf:
                max_perf=perf
                best_performing_sym = ticker
        except:
            pass
    print("Best opportunity for spot/future arb with "+best_performing_sym + " with "+str(round(100*max_perf,2))+"% return")
    return best_performing_sym

def get_all_future_tickers():
    tickers = []
    for instrument in json.loads(apif.get_instruments())["instruments"]:
        sym = instrument["symbol"]
        if len(sym.split("_"))==3:
            tickers.append(sym.split("_")[1])
    return list(set(tickers))

def get_future_expiry(ticker_future):
    future_expiry_info = ticker_future.split("_")[2]
    y = 2000 + int(future_expiry_info[:2])
    m = int(future_expiry_info[2:4].replace("0",""))
    d = int(future_expiry_info[4:].replace("0",""))
    return datetime.datetime(y,m,d)

def format_ticker_for_spot_api(ticker):
    return "X" + ticker.upper()[:3] + "Z" + ticker.upper()[3:]


def compute_spot_fut_arb_performance_monthly(ticker, notional_usd, debug=True):
    ticker_future = apif.get_ticker("fi_"+ticker, "M")
    price_future = apif.get_bid(ticker_future)
    price_spot = apis.get_ask(format_ticker_for_spot_api(ticker))
    # buy btc spot
    notional_crypto = notional_usd/price_spot
    # short btc fut for same notional
    trade_profit_future = notional_crypto*price_future
    profit = trade_profit_future-notional_usd
    days_to_expiry = (get_future_expiry(ticker_future) - datetime.datetime.now()).days
    y = profit/notional_usd
    if debug:
        print("Performance of "+ticker+" spot/future arb strategy:")
        print("    Profit: $", profit )
        print("    Yield: ",round(y*100,2), "%")
        print("    Equivalent annual yield: ",round(365/days_to_expiry*y*100,2), "%")
        print("-----------------------------------------------------------------------")
    return y

def compute_spot_fut_arb_performance_quarterly(ticker, notional_usd, debug=True):
    """
    ticker: str, ethusd
    """
    ticker_future = apif.get_ticker("fi_"+ticker, "Q")
    price_future = apif.get_bid(ticker_future)
    price_spot = apis.get_ask(format_ticker_for_spot_api(ticker))
    # buy btc spot
    notional_crypto = notional_usd/price_spot
    # short btc fut for same notional
    trade_profit_future = notional_crypto*price_future
    profit = trade_profit_future-notional_usd
    days_to_expiry = (get_future_expiry(ticker_future) - datetime.datetime.now()).days
    y = profit/notional_usd
    if debug:
        print("Performance of "+ticker+" spot/future arb strategy:")
        print("    Profit: $", profit )
        print("    Yield: ",round(y*100,2), "%")
        print("    Equivalent annual yield: ",round(365/days_to_expiry*y*100,2), "%")
        print("-----------------------------------------------------------------------")
    return y

def execute_spot_fut_arb(ticker, notional):
    """
    ticker: str, "xbtusd"
    """
    spot_ticker = format_ticker_for_spot_api(ticker)
    future_ticker = apif.get_ticker("fi_"+ticker, "M")
    
    # check we have the notional on kraken spot
    status="not done"
    if float(apis.get_balance("ZUSD"))*1.1>notional:
        # buy the notional in ticker
        spot_p = apis.get_mid("XXBTZUSD")
        notional_crypto = notional/spot_p
        res = apis.create_order(spot_ticker,"buy","limit",spot_p,notional_crypto)
        if res["error"]==[]:
            order_id = res["txid"]
        # check order exectuted
        # send the notional to kraken futures
        apis.transfer_from_spot_to_future("XXBT", )
        return status
    else:
        raise ValueError("Not enough USD!")
    
    # short the future
    return status

def get_open_orders_id():
    open_orders = []
    res = apis.query_private("OpenOrders")
    for order in res["result"]['open']:
        open_orders.append(order)
    return open_orders

def get_closed_orders_id():
    closed_orders = []
    res = apis.query_private("ClosedOrders")
    for order in res["result"]['closed']:
        closed_orders.append(order)
    return closed_orders

def get_lhs(ticker):
    i = int(len(ticker)/2)
    return ticker[:i]

def get_rhs(ticker):
    i = int(len(ticker)/2)
    return ticker[i:]

def get_future_balance(ticker):
    t = "fi_"+ticker
    return json.loads(apif.get_accounts())["accounts"][t]

def get_leverage(future_ticker):
    account = json.loads(apif.get_accounts())["accounts"]["_".join(future_ticker.split("_")[:2])]
    size = account["balances"][future_ticker]
    pv = account["auxiliary"]['pv']
    mtmPrice = apif.get_mid(future_ticker)
    return abs(size)/(pv*mtmPrice)

def get_size_to_short_to_get_leverage_to_one(future_ticker):
    account = json.loads(apif.get_accounts())["accounts"]["_".join(future_ticker.split("_")[:2])]
    size = account["balances"][future_ticker]
    pv = account["auxiliary"]['pv']
    mtmPrice = apif.get_mid(future_ticker)
    return int(pv*mtmPrice - abs(size))

def create_kraken_api(api_type,account):
    with open("C:/dev/data/kraken/k"+account+"_"+api_type+".txt") as f:
        content = f.readlines()
    public_key = content[0][:-1]
    private_key = content[1]
    if api_type=="spot":
        return API_spot(apiPublicKey=public_key, apiPrivateKey=private_key) 
    else:
        return API_future(apiPublicKey=public_key, apiPrivateKey=private_key)

In [16]:
apis = create_kraken_api("spot","archi")
apif = create_kraken_api("future","archi")

In [17]:
apis.get_balance("ZUSD")

'0.0000'

In [162]:
apis.get_balance("ZEUR")

'100.0000'

In [18]:
ticker = get_best_opportunity_ticker_arb_future_spot()

Performance of xrpusd spot/future arb strategy:
    Profit: $ 30.180130804925284
    Yield:  3.02 %
    Equivalent annual yield:  550.79 %
-----------------------------------------------------------------------
Performance of xbtusd spot/future arb strategy:
    Profit: $ 22.370970887021826
    Yield:  2.24 %
    Equivalent annual yield:  408.27 %
-----------------------------------------------------------------------
Performance of ltcusd spot/future arb strategy:
    Profit: $ 37.16322465667258
    Yield:  3.72 %
    Equivalent annual yield:  678.23 %
-----------------------------------------------------------------------
Performance of ethusd spot/future arb strategy:
    Profit: $ 32.8101366812009
    Yield:  3.28 %
    Equivalent annual yield:  598.78 %
-----------------------------------------------------------------------
Best opportunity for spot/future arb with ltcusd with 3.72% return


### SETUP

In [164]:
apif.get_ticker("fi_"+ticker, "M")

'fi_ltcusd_210430'

### Check we are at 1.0 leverage

In [ ]:
leverage=get_leverage(future_ticker)
if leverage>1.005 or leverage<0.995:
    raise ValueError("LEVERAGE SHOULD BE ONE, WHAT FUCKING HAPPENED ARTHUR?")
else:
    print("Leverage is fine:",leverage)

In [19]:
# We don't want to to vig trades so 1k at a time max
notional = min(1000,float(apis.get_balance("ZUSD")))
# cancel the order if not executed after 30s (market may have moved)
time_to_expiry_in_seconds = 30
print("Starting to execute spot/future arb for "+ticker+ " with notional = ", get_rhs(ticker).upper(), notional)
spot_ticker = format_ticker_for_spot_api(ticker)
future_ticker = apif.get_ticker("fi_"+ticker, "M")
print("The future we're going to short is:",future_ticker)
spot_ask = apis.get_ask(spot_ticker)
future_bid = apif.get_bid(future_ticker)
print("The expected return is:", round(100*(future_bid-spot_ask)/spot_ask,2), "%")

Starting to execute spot/future arb for ltcusd with notional =  USD 0.0
The future we're going to short is: fi_ltcusd_210430
The expected return is: 3.68 %


### Buy spot

In [203]:
spot_p = round(apis.get_mid(spot_ticker),2)
notional_crypto = notional/spot_p
limitPrice = apif.get_mid(future_ticker)
qty =  int(notional_crypto*limitPrice)
qty

533

In [207]:
# check we have the notional on kraken spot
status="not done"
trials_nb = 5
i=0
initial_crypto_balance = apis.get_balance("X"+get_lhs(ticker).upper())
while (apis.get_balance("X"+get_lhs(ticker).upper())<=initial_crypto_balance) & (i<trials_nb):
    i+=1
    if float(apis.get_balance("ZUSD"))*1.1>notional:

        # buy the notional in ticker
        spot_p = round(apis.get_mid(spot_ticker),1)
        notional_crypto = notional/spot_p
        print("Sending a buy order for "+ticker.upper() + " @",spot_p, "resulting in", notional_crypto, get_lhs(ticker).upper())
        trade_time = datetime.datetime.now()
        res = apis.create_order(spot_ticker,"buy","limit",spot_p,notional_crypto)
        expiry_time = trade_time+datetime.timedelta(time_to_expiry_in_seconds*1/24/60/60)
        print(res)
        order_id = res['result']['txid'][0]
        print("Created order:",order_id)

        # check order exectuted
        status = "open" 
        expired = False
        while status== "open" and not expired:
            try:
                time.sleep(3)
                order_query = apis.query_private("QueryOrders",{"txid":order_id})
                status = order_query["result"][order_id]["status"]
            except Exception as e:
                print("Error:",e)
            expired = expiry_time<datetime.datetime.now()
        if expired:
            # cancel trade
            print("Failed to buy within 30s, aborting strategy.")
            apis.query_private("CancelOrder", {"txid":order_id})
            order_query = apis.query_private("QueryOrders",{"txid":order_id})
            status = order_query["result"][order_id]["status"]    
            if status!="cancelled":
                raise ValueError("ORDER FAILED TO CANCEL")
        else:
            print("Succesfully bought",get_lhs(ticker).upper(),notional_crypto)
            print("status: ",status)
    else:
        print("Not enough balance, aborting strategy.")

Sending a buy order for LTCUSD @ 184.1 resulting in 2.830162411732754 LTC
{'error': [], 'result': {'descr': {'order': 'buy 2.83016241 LTCUSD @ limit 184.10'}, 'txid': ['OIN7XO-OXOF6-RAQIB5']}}
Created order: OIN7XO-OXOF6-RAQIB5
Succesfully bought LTC 2.830162411732754
status:  canceled


## Transfer to fut

In [211]:
#notional_crypto = float(apis.get_balance("X"+get_lhs(ticker).upper()))
notional_crypto = floor(notional_crypto*100)/100#round(notional_crypto,2)
# transfer from spot to future portfolio
print("Transfering ", get_lhs(ticker).upper(),notional_crypto, "from spot to future portfolio...")
future_balance_init = apif.get_cash(get_lhs(ticker))
print("   initial balance on future account:",future_balance_init)
future_balance = future_balance_init
res = apis.transfer_from_spot_to_future(spot_ticker[:4],notional_crypto)
print(res)
transfer_id = res["result"]['refid']

while future_balance < future_balance_init + float(notional_crypto):
    future_balance = apif.get_cash(get_lhs(ticker))
print("... transfer from SPOT to FUTURE done, new balance:",future_balance)


# transfer from wallet to specific ticker portfolio
notional_crypto = apif.get_cash(get_lhs(ticker))

print("Transfering",get_lhs(ticker).upper(),notional_crypto, "from cash wallet to future ("+"fi_"+ticker+") portfolio...")
apif.transfer("cash", "fi_"+ticker, get_lhs(ticker), notional_crypto)
while future_balance > future_balance_init :
    future_balance = apif.get_cash(get_lhs(ticker))
    time.sleep(5)
print("... transfer from cash wallet to specific future portfolio done.")

Transfering  LTC 2.83 from spot to future portfolio...
   initial balance on future account: 0.0
{'error': [], 'result': {'refid': 'BOFAAIR-JVY53T-TAJ277'}}


## Short future

In [220]:
# create order
trade_done= False
i=0
nb_trials = 5
while (not trade_done) and i<nb_trials:
    i+=1
    limitPrice = round(apif.get_mid(future_ticker),4)
    qty =  int(notional_crypto*limitPrice)+1
    print("Shorting future:"+future_ticker,", qty:",get_rhs(ticker).upper(),qty, ", price:",limitPrice)
    trade_time = datetime.datetime.now()
    res = json.loads(apif.send_order("lmt", future_ticker, "sell", qty, round(limitPrice,2)))
    expiry_time = trade_time+datetime.timedelta(time_to_expiry_in_seconds*1/24/60/60)
    order_id = res["sendStatus"]["order_id"]
    print("order_id",order_id)

    # wait for it to be done
    is_open = True 
    expired = False
    while is_open and not expired: 
        openOrders = json.loads(apif.get_openorders())["openOrders"]
        if len(openOrders)>0:
            is_open = order_id == openOrders[0]["order_id"]
        expired = expiry_time<datetime.datetime.now()
    if expired:
        # cancel trade
        print("Failed to buy within 30s, cancelling order, aborting strategy.")
        apif.cancel_order(order_id)
        openOrders = json.loads(apif.get_openorders())["openOrders"]
        if order_id in openOrders:
            raise ValueError("ORDER FAILED TO CANCEL")
    else:
        print("Succesfully shorted")
        trade_done = True

Shorting future:fi_ltcusd_210430 , qty: USD 534 , price: 188.68
order_id 97729408-9481-4a17-b248-002cb0541c1d


In [221]:
res

{'result': 'success',
 'sendStatus': {'orderEvents': [{'order': {'algoId': None,
     'cliOrdId': None,
     'filled': 0,
     'lastUpdateTimestamp': '2021-03-27T21:34:37.821Z',
     'limitPrice': 188.68,
     'orderId': '97729408-9481-4a17-b248-002cb0541c1d',
     'quantity': 534,
     'reduceOnly': False,
     'side': 'sell',
     'symbol': 'fi_ltcusd_210430',
     'timestamp': '2021-03-27T21:34:37.821Z',
     'type': 'lmt'},
    'reducedQuantity': None,
    'type': 'PLACE'}],
  'order_id': '97729408-9481-4a17-b248-002cb0541c1d',
  'receivedTime': '2021-03-27T21:34:37.821Z',
  'status': 'placed'},
 'serverTime': '2021-03-27T21:34:37.822Z'}

### Are we clear on leverage ? Else get leverage to 1.0

In [32]:
leverage=get_leverage(future_ticker)
if leverage>1.005 or leverage<0.995:
    size_to_short = get_size_to_short_to_get_leverage_to_one(future_ticker)
    print("You should short "+ str(size_to_short) +" more contracts")
    # create order
    trade_done= False
    i=0
    nb_trials = 5
    while (not trade_done) and i<nb_trials:
        i+=1
        limitPrice = round(apif.get_mid(future_ticker),4)
        qty =  abs(size_to_short)
        buy_sell = "sell" if size_to_short<0 else "buy"
        print("Shorting future:"+future_ticker,", qty:",get_rhs(ticker).upper(),qty, ", price:",limitPrice)
        trade_time = datetime.datetime.now()
        res = json.loads(apif.send_order("lmt", future_ticker, buy_sell, qty, round(limitPrice,2)))
        expiry_time = trade_time+datetime.timedelta(time_to_expiry_in_seconds*1/24/60/60)
        order_id = res["sendStatus"]["order_id"]
        print("order_id",order_id)

        # wait for it to be done
        is_open = True 
        expired = False
        while is_open and not expired: 
            openOrders = json.loads(apif.get_openorders())["openOrders"]
            if len(openOrders)>0:
                is_open = order_id == openOrders[0]["order_id"]
            expired = expiry_time<datetime.datetime.now()
        if expired:
            # cancel trade
            print("Failed to buy within 30s, cancelling order, aborting strategy.")
            apif.cancel_order(order_id)
            openOrders = json.loads(apif.get_openorders())["openOrders"]
            if order_id in openOrders:
                raise ValueError("ORDER FAILED TO CANCEL")
        else:
            print("Succesfully shorted")
            trade_done = True
        
else:
    print("Leverage is fine:",leverage)

Shorting future:fi_xbtusd_210924 , qty: USD 120 , price: 53569.0
order_id a0582004-a831-4c0d-83fb-470e21e57ac6


In [118]:
res

{'result': 'success',
 'sendStatus': {'orderEvents': [{'order': {'algoId': None,
     'cliOrdId': None,
     'filled': 0,
     'lastUpdateTimestamp': '2021-03-16T11:27:05.871Z',
     'limitPrice': 0.48,
     'orderId': 'f2ca845e-7ea5-49a9-b572-59021f2d450f',
     'quantity': 954,
     'reduceOnly': False,
     'side': 'sell',
     'symbol': 'fi_xrpusd_210326',
     'timestamp': '2021-03-16T11:27:05.871Z',
     'type': 'lmt'},
    'reducedQuantity': None,
    'type': 'PLACE'}],
  'order_id': 'f2ca845e-7ea5-49a9-b572-59021f2d450f',
  'receivedTime': '2021-03-16T11:27:05.871Z',
  'status': 'placed'},
 'serverTime': '2021-03-16T11:27:05.874Z'}

In [119]:
get_leverage(future_ticker)

0.5020863454558158

## Define when to close pos ?

In [ ]:
# when to close order?
# if order closed, get all the eth from futures to general wallet
#then from future to spot wallet
apif.send_withdrawal_to_spot(get_lhs(ticker), 0.01)

### recent orders

In [333]:
for order in json.loads(apif.get_recentorders("FI_ETHUSD_210129"))["orderEvents"]:
    print("K",order["event"].keys())
    try:
        print("ID", order["event"]['orderPlaced']["order"]["uid"])
    except:
        try:
            print(order["event"]['executionEvent'])
        except:
            print(order["event"]['orderRejected'])
    print("----")

K dict_keys(['timestamp', 'uid', 'orderPlaced'])
ID db54ff1f-2ab2-4743-afa5-9a74d145e8e9
----
K dict_keys(['timestamp', 'uid', 'executionEvent'])
{'execution': {'uid': 'ed568211-fc6c-4c2d-82a7-c4e8c13bd206', 'takerOrder': {'uid': 'db54ff1f-2ab2-4743-afa5-9a74d145e8e9', 'accountId': '113446', 'tradeable': 'FI_ETHUSD_210129', 'direction': 'SELL', 'quantity': '40', 'filled': '0', 'timestamp': '1610636123969', 'limitPrice': '1214.65', 'orderType': 'IMMEDIATE_OR_CANCEL', 'clientId': '', 'stopPrice': '', 'reduceOnly': False}, 'timestamp': '1610636123969', 'quantity': '40', 'price': '1226.9', 'markPrice': '1227.82500000000', 'limitFilled': False}, 'takerReducedQuantity': ''}
----
K dict_keys(['timestamp', 'uid', 'orderRejected'])
{'order': {'uid': '552c3728-c7f5-4d90-a0e9-24a143107dc9', 'accountId': '113446', 'tradeable': 'FI_ETHUSD_210129', 'direction': 'SELL', 'quantity': '50', 'filled': '0', 'timestamp': '1610636208701', 'limitPrice': '1229.175', 'orderType': 'LIMIT', 'clientId': '', 'stop

'{"result":"success","serverTime":"2021-01-14T12:25:35.572Z"}'